Análisis No Supervisado con K-Means sobre el Dataset de Scoring Crediticio

Autor: Eduardo Molina-Diego Martínez

Curso: Machine Learning – Examen transversal

Técnica aplicada: K-Means Clustering

## Introducción

En esta sección se aplica un método de aprendizaje no supervisado como parte del proyecto de scoring crediticio.
El objetivo es identificar segmentos homogéneos de clientes utilizando K-Means, y posteriormente analizar si
estos grupos presentan diferencias relevantes en su comportamiento crediticio, especialmente en la variable TARGET.

El análisis se realiza estrictamente sobre el conjunto de entrenamiento, cumpliendo con el requisito de evitar
cualquier forma de data leakage. A través del enfoque CRISP-DM, se detallan las fases de business understanding,
data understanding, preparación, modelado, evaluación e interpretación de resultados.


## Técnica seleccionada

Se utiliza **K-Means**, un algoritmo de clustering particional que permite segmentar clientes en grupos
basados en similitud de características. Este método es adecuado para este problema porque:

- Funciona bien con grandes volúmenes de datos.
- Permite descubrir patrones ocultos sin necesidad de etiquetas.
- Facilita el análisis posterior de riesgo por segmento comparando contra TARGET.
- Puede incorporarse como variable adicional en un modelo supervisado o como herramienta de negocio.

Las variables utilizadas provienen solo del conjunto de entrenamiento (`application_train`).


## CRISP–DM

### 1. Business Understanding
El objetivo es mejorar la comprensión del conjunto de clientes mediante segmentación, buscando patrones o
subpoblaciones con comportamiento crediticio distinto. Esto ayuda a identificar posibles sesgos, riesgos emergentes
y fortalece al modelo supervisado principal.

### 2. Data Understanding
Se utilizó únicamente el dataset `application_train`. Se exploraron valores faltantes, distribución de variables,
rangos y correlaciones. Se crearon nuevas variables relevantes para representar edad, antigüedad laboral
y ratios de endeudamiento.

### 3. Data Preparation
- Imputación por mediana.
- Escalamiento con StandardScaler.
- Selección de variables numéricas.
- Creación de variables derivadas: AGE, EMPLOYED_YEARS, CREDIT_INCOME_RATIO, ANNUITY_INCOME_RATIO.

### 4. Modeling
Se evaluaron distintos valores de k usando:
- Método del codo (inercia).
- Silhouette Score.

El k óptimo se utilizó para entrenar el modelo K-Means final.

### 5. Evaluation
Se interpretaron los clusters mediante:
- Perfiles promedio por cluster.
- Distribuciones en PCA 2D.
- Tasa de TARGET=1 por cluster.

Esto permitió evaluar si los clusters representan segmentos con distinto riesgo crediticio.

### 6. Deployment (discusión)
El número de cluster puede incorporarse como feature adicional, o servir como herramienta de monitoreo
de subpoblaciones con riesgo distinto.


In [ ]:
# ============================================================
# Imports y configuración
# ============================================================

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (10, 5)
sns.set(style="whitegrid")


In [ ]:
!pip install gdown -q

import gdown
import pandas as pd

# ID del archivo
file_id = "1kxIFtFJcDxTgSeUR0LA5yFEG8lYuuBej"

# URL directa para gdown
url = f"https://drive.google.com/uc?id={file_id}"

output = "application_.parquet"

# Descargar archivo
gdown.download(url, output, quiet=False)

# Leer archivo parquet
df = pd.read_parquet(output)

df.head()


Downloading...
From: https://drive.google.com/uc?id=1kxIFtFJcDxTgSeUR0LA5yFEG8lYuuBej
To: /content/application_.parquet
100%|██████████| 22.2M/22.2M [00:00<00:00, 48.2MB/s]


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## Integración de múltiples fuentes (Examen) — Construcción de `df_full`

En esta sección se integran tablas adicionales del dataset Home Credit para generar una tabla final por cliente (`SK_ID_CURR`).
Este bloque **no reemplaza** tu lógica posterior: simplemente construye `df_full` y luego asigna `df = df_train` para mantener el análisis no supervisado sin data leakage.


In [ ]:
import os
import numpy as np
import pandas as pd

def safe_div(a, b):
    return np.where((b == 0) | (pd.isna(b)), np.nan, a / b)

def flatten_cols(df_agg):
    df_agg.columns = [
        f"{c[0]}__{c[1]}" if isinstance(c, tuple) else str(c)
        for c in df_agg.columns
    ]
    return df_agg

# ---- Configuración de ruta base ----
# Si tus parquet están en otra carpeta, cambia BASE_DIR (por ejemplo a tu carpeta de Drive)
BASE_DIR = os.environ.get("HOME_CREDIT_PARQUET_DIR", ".")
print("BASE_DIR:", BASE_DIR)

required_cols = {"SK_ID_CURR", "TARGET"}
assert required_cols.issubset(set(df.columns)), "df debe contener SK_ID_CURR y TARGET."

application = df.copy()

paths = {
    "bureau": "bureau.parquet",
    "bureau_balance": "bureau_balance.parquet",
    "previous_application": "previous_application.parquet",
    "pos_cash": "POS_CASH_balance.parquet",
    "installments": "installments_payments.parquet",
    "credit_card": "credit_card_balance.parquet",
}

def rp(name):
    p = os.path.join(BASE_DIR, paths[name])
    return pd.read_parquet(p)

bureau = rp("bureau")
bureau_balance = rp("bureau_balance")
previous_application = rp("previous_application")
pos_cash = rp("pos_cash")
installments = rp("installments")
credit_card = rp("credit_card")

print("application:", application.shape)
print("bureau:", bureau.shape)
print("bureau_balance:", bureau_balance.shape)
print("previous_application:", previous_application.shape)
print("pos_cash:", pos_cash.shape)
print("installments:", installments.shape)
print("credit_card:", credit_card.shape)


## Data Understanding (Análisis Exploratorio Inicial)

Antes de entrenar el modelo, se realiza un análisis exploratorio básico del dataset integrado (`df_full`)
con el objetivo de:

- Comprender la distribución de la variable objetivo (TARGET)
- Identificar posibles desbalances de clases
- Detectar valores faltantes y rangos atípicos
- Validar que la integración de múltiples fuentes no haya introducido inconsistencias

Este análisis se utiliza únicamente para comprensión del problema y no influye directamente en el entrenamiento.



In [ ]:
# Distribución de la variable objetivo
df_full["TARGET"].value_counts(normalize=True)


import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x="TARGET", data=df_full)
plt.title("Distribución de la variable objetivo (TARGET)")
plt.show()


NameError: name 'df_full' is not defined

## Evaluación de Calidad de Datos

Se revisa la presencia de valores faltantes en el dataset final integrado.  
El objetivo es identificar variables que requieren imputación o tratamiento especial
antes del entrenamiento del modelo.


In [ ]:
# Porcentaje de valores nulos por columna (top 15)
missing_pct = (
    df_full.isnull()
    .mean()
    .sort_values(ascending=False)
    .head(15)
)

missing_pct


## Justificación del Preprocesamiento

Debido a la integración de múltiples fuentes de datos, es esperable la presencia de valores faltantes,
ya que no todos los clientes poseen historial crediticio, tarjetas o créditos previos.

Por esta razón:
- Se utiliza imputación (por mediana o estrategia definida en el pipeline)
- Se evita la eliminación masiva de registros para no perder información
- Se mantiene la coherencia entre conjuntos de entrenamiento, validación y prueba

Estas decisiones permiten entrenar un modelo más robusto y representativo del escenario real.


### 1) bureau_balance → agregación por `SK_ID_BUREAU` y merge a bureau


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x="TARGET", data=df_full)
plt.title("Distribución de la variable objetivo (TARGET)")
plt.show()
bb_agg = (
    bureau_balance
    .groupby("SK_ID_BUREAU")
    .agg(
        bb_months_count=("MONTHS_BALANCE", "count"),
        bb_months_min=("MONTHS_BALANCE", "min"),
        bb_months_max=("MONTHS_BALANCE", "max"),
        bb_status_mean=("STATUS", lambda x: pd.to_numeric(x, errors="coerce").mean())
    )
    .reset_index()
)

bureau_bb = bureau.merge(bb_agg, on="SK_ID_BUREAU", how="left")
print("bureau_bb:", bureau_bb.shape)


### 2) bureau → agregación por cliente `SK_ID_CURR`


In [ ]:
bureau_cols = [c for c in [
    "AMT_CREDIT_SUM",
    "AMT_CREDIT_SUM_DEBT",
    "AMT_CREDIT_SUM_OVERDUE",
    "AMT_CREDIT_MAX_OVERDUE",
    "DAYS_CREDIT",
    "CREDIT_DAY_OVERDUE",
    "DAYS_CREDIT_ENDDATE",
    "bb_months_count",
    "bb_months_min",
    "bb_months_max",
] if c in bureau_bb.columns]

agg_dict = {c: ["mean", "max", "min", "sum"] for c in bureau_cols}

if "CREDIT_ACTIVE" in bureau_bb.columns:
    active_dummies = pd.get_dummies(bureau_bb["CREDIT_ACTIVE"], prefix="bureau_active")
    bureau_bb2 = pd.concat([bureau_bb[["SK_ID_CURR"]], bureau_bb[bureau_cols], active_dummies], axis=1)
    for c in active_dummies.columns:
        agg_dict[c] = ["sum"]
else:
    bureau_bb2 = bureau_bb[["SK_ID_CURR"] + bureau_cols].copy()

bureau_agg = bureau_bb2.groupby("SK_ID_CURR").agg(agg_dict)
bureau_agg = flatten_cols(bureau_agg).reset_index()

if "AMT_CREDIT_SUM__sum" in bureau_agg.columns and "AMT_CREDIT_SUM_DEBT__sum" in bureau_agg.columns:
    bureau_agg["bureau_debt_to_credit_sum_ratio"] = safe_div(
        bureau_agg["AMT_CREDIT_SUM_DEBT__sum"],
        bureau_agg["AMT_CREDIT_SUM__sum"]
    )

bureau_agg.head()


### 3) previous_application → agregación por cliente `SK_ID_CURR`


In [ ]:
pa = previous_application.copy()

if "NAME_CONTRACT_STATUS" in pa.columns:
    pa["prev_is_approved"] = (pa["NAME_CONTRACT_STATUS"] == "Approved").astype(int)
    pa["prev_is_refused"] = (pa["NAME_CONTRACT_STATUS"] == "Refused").astype(int)

pa_cols = [c for c in [
    "AMT_APPLICATION",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "AMT_DOWN_PAYMENT",
    "DAYS_DECISION"
] if c in pa.columns]

pa_agg_dict = {c: ["mean", "max", "min", "sum"] for c in pa_cols}

for c in ["prev_is_approved", "prev_is_refused"]:
    if c in pa.columns:
        pa_agg_dict[c] = ["sum", "mean"]

prev_agg = pa.groupby("SK_ID_CURR").agg(pa_agg_dict)
prev_agg = flatten_cols(prev_agg).reset_index()

if "prev_is_approved__sum" in prev_agg.columns and "prev_is_refused__sum" in prev_agg.columns:
    denom = prev_agg["prev_is_approved__sum"] + prev_agg["prev_is_refused__sum"]
    prev_agg["prev_approval_rate"] = safe_div(prev_agg["prev_is_approved__sum"], denom)

prev_agg.head()


### 4) POS_CASH_balance → agregación por cliente `SK_ID_CURR`


In [ ]:
pos = pos_cash.copy()

pos_cols = [c for c in [
    "MONTHS_BALANCE",
    "CNT_INSTALMENT",
    "CNT_INSTALMENT_FUTURE",
    "SK_DPD",
    "SK_DPD_DEF"
] if c in pos.columns]

pos_agg_dict = {c: ["mean", "max", "min", "sum"] for c in pos_cols}

pos_agg = pos.groupby("SK_ID_CURR").agg(pos_agg_dict)
pos_agg = flatten_cols(pos_agg).reset_index()

pos_agg.head()


### 5) installments_payments → agregación por cliente `SK_ID_CURR`


In [ ]:
inst = installments.copy()

if "DAYS_ENTRY_PAYMENT" in inst.columns and "DAYS_INSTALMENT" in inst.columns:
    inst["days_late"] = inst["DAYS_ENTRY_PAYMENT"] - inst["DAYS_INSTALMENT"]
    inst["is_late"] = (inst["days_late"] > 0).astype(int)
else:
    inst["days_late"] = np.nan
    inst["is_late"] = np.nan

inst_cols = [c for c in [
    "AMT_INSTALMENT",
    "AMT_PAYMENT",
    "days_late",
    "is_late"
] if c in inst.columns]

inst_agg_dict = {c: ["mean", "max", "min", "sum"] for c in inst_cols}

inst_agg = inst.groupby("SK_ID_CURR").agg(inst_agg_dict)
inst_agg = flatten_cols(inst_agg).reset_index()

if "AMT_PAYMENT__sum" in inst_agg.columns and "AMT_INSTALMENT__sum" in inst_agg.columns:
    inst_agg["install_pay_ratio_sum"] = safe_div(
        inst_agg["AMT_PAYMENT__sum"], inst_agg["AMT_INSTALMENT__sum"]
    )

inst_agg.head()


### 6) credit_card_balance → agregación por cliente `SK_ID_CURR`


In [ ]:
cc = credit_card.copy()

cc_cols = [c for c in [
    "MONTHS_BALANCE",
    "AMT_BALANCE",
    "AMT_CREDIT_LIMIT_ACTUAL",
    "AMT_DRAWINGS_ATM_CURRENT",
    "AMT_DRAWINGS_CURRENT",
    "AMT_PAYMENT_CURRENT",
    "SK_DPD",
    "SK_DPD_DEF"
] if c in cc.columns]

cc_agg_dict = {c: ["mean", "max", "min", "sum"] for c in cc_cols}

cc_agg = cc.groupby("SK_ID_CURR").agg(cc_agg_dict)
cc_agg = flatten_cols(cc_agg).reset_index()

if "AMT_BALANCE__mean" in cc_agg.columns and "AMT_CREDIT_LIMIT_ACTUAL__mean" in cc_agg.columns:
    cc_agg["cc_utilization_mean"] = safe_div(
        cc_agg["AMT_BALANCE__mean"], cc_agg["AMT_CREDIT_LIMIT_ACTUAL__mean"]
    )

cc_agg.head()


### 7) Merge final 1:1 por cliente → `df_full`


In [ ]:
df_full = application.copy()

for feat_df, name in [
    (bureau_agg, "bureau_agg"),
    (prev_agg, "prev_agg"),
    (pos_agg, "pos_agg"),
    (inst_agg, "inst_agg"),
    (cc_agg, "cc_agg"),
]:
    before = df_full.shape[0]
    df_full = df_full.merge(feat_df, on="SK_ID_CURR", how="left")
    after = df_full.shape[0]
    assert before == after, f"Merge con {name} cambió filas ({before} -> {after})."

print("df_full shape:", df_full.shape)
df_full.head()


## Split Train/Val/Test (data leakage) y continuidad del notebook

Para mantener tu notebook sin modificar celdas posteriores, a partir de aquí se define:
- `df_all = df_full`
- `df_train, df_val, df_test`
- `df = df_train` (para que el resto del notebook opere solo con TRAIN)


In [ ]:
from sklearn.model_selection import train_test_split

df_all = df_full.copy()

df_tmp, df_test = train_test_split(
    df_all, test_size=0.20, stratify=df_all["TARGET"], random_state=42
)
df_train, df_val = train_test_split(
    df_tmp, test_size=0.25, stratify=df_tmp["TARGET"], random_state=42
)

print("Train:", df_train.shape, " Val:", df_val.shape, " Test:", df_test.shape)

# Reasignamos df para que el resto del notebook (clustering) use SOLO TRAIN
df = df_train.copy()


In [ ]:
# ============================================================
# 2. Creación de variables derivadas
# ============================================================

# Edad en años (los DAYS_BIRTH vienen negativos)
df["AGE"] = -df["DAYS_BIRTH"] / 365

# Años de empleo (también negativo en el dataset original)
df["EMPLOYED_YEARS"] = -df["DAYS_EMPLOYED"] / 365

# Ratios de riesgo financiero
df["CREDIT_INCOME_RATIO"] = df["AMT_CREDIT"] / df["AMT_INCOME_TOTAL"]
df["ANNUITY_INCOME_RATIO"] = df["AMT_ANNUITY"] / df["AMT_INCOME_TOTAL"]

# Lista de variables que usaremos para el clustering
FEATURES = [
    "AMT_INCOME_TOTAL",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "AGE",
    "EMPLOYED_YEARS",
    "CREDIT_INCOME_RATIO",
    "ANNUITY_INCOME_RATIO"
]

df_clustering = df[FEATURES].copy()
df_clustering.describe()


,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AGE,EMPLOYED_YEARS,CREDIT_INCOME_RATIO,ANNUITY_INCOME_RATIO
count,3.075110e+05,3.075110e+05,307499.000000,307511.000000,307511.000000,307511.000000,307499.000000
mean,1.687979e+05,5.990260e+05,27108.573909,43.936973,-174.835742,3.957570,0.180930
std,2.371231e+05,4.024908e+05,14493.737315,11.956133,387.056895,2.689728,0.094574
min,2.565000e+04,4.500000e+04,1615.500000,20.517808,-1000.665753,0.004808,0.000224
25%,1.125000e+05,2.700000e+05,16524.000000,34.008219,0.791781,2.018667,0.114782
50%,1.471500e+05,5.135310e+05,24903.000000,43.150685,3.323288,3.265067,0.162833
75%,2.025000e+05,8.086500e+05,34596.000000,53.923288,7.561644,5.159880,0.229067
max,1.170000e+08,4.050000e+06,258025.500000,69.120548,49.073973,84.736842,1.875965


In [ ]:
# ============================================================
# 3. Imputación de valores faltantes y escalamiento
# ============================================================

# Imputación por mediana
for col in FEATURES:
    median_val = df_clustering[col].median()
    df_clustering[col] = df_clustering[col].fillna(median_val)

# Escalamiento estándar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clustering)

df_scaled = pd.DataFrame(X_scaled, columns=FEATURES)
df_scaled.head()


,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AGE,EMPLOYED_YEARS,CREDIT_INCOME_RATIO,ANNUITY_INCOME_RATIO
0,0.142129,-0.478095,-0.166143,-1.506880,0.456215,-0.724863,-0.623352
1,0.426792,1.725450,0.592683,0.166821,0.460115,0.309764,-0.515086
2,-0.427196,-1.152888,-1.404669,0.689509,0.453299,-0.727796,-0.855744
3,-0.142533,-0.711430,0.177874,0.680114,0.473217,-0.610250,0.412075
4,-0.199466,-0.213734,-0.361749,0.892535,0.473210,0.098394,-0.010218


In [ ]:
# ============================================================
# 4. Búsqueda del número óptimo de clusters (k)
#    Usando método del codo + silhouette
# ============================================================

k_values = range(2, 11)
inertias = []
sil_scores = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(df_scaled)

    inertias.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(df_scaled, labels))

# Gráfico del codo
plt.figure()
plt.plot(k_values, inertias, marker="o")
plt.xlabel("Número de clusters (k)")
plt.ylabel("Inercia")
plt.title("Método del codo")
plt.show()

# Gráfico de silhouette
plt.figure()
plt.plot(k_values, sil_scores, marker="o")
plt.xlabel("Número de clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score por k")
plt.show()

# Seleccionar k con mejor silhouette
optimal_k = k_values[np.argmax(sil_scores)]
print("Mejor k según Silhouette Score:", optimal_k)


KeyboardInterrupt: 

In [ ]:
# ============================================================
# 5. Entrenamiento final de K-Means con k óptimo
# ============================================================

kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(df_scaled)

df["CLUSTER"] = cluster_labels
df_clustering["CLUSTER"] = cluster_labels

df[["CLUSTER"] + FEATURES].head()


In [ ]:
# ============================================================
# 6. Análisis de perfiles por cluster
# ============================================================

cluster_profiles = df_clustering.groupby("CLUSTER")[FEATURES].mean()
print("Perfiles promedio de cada cluster:")
display(cluster_profiles)

# Si quieres ver cuántas observaciones hay por cluster:
cluster_counts = df["CLUSTER"].value_counts().sort_index()
print("\nCantidad de clientes por cluster:")
print(cluster_counts)


In [ ]:
# ============================================================
# 7. Relación entre clusters y riesgo
# ============================================================


assert "TARGET" in df.columns, "No se encontró la columna TARGET en el dataframe."

# Tasa de morosidad promedio por cluster
default_rates = df.groupby("CLUSTER")["TARGET"].mean().sort_index()

print("Tasa de TARGET=1 por cluster:")
print(default_rates)

plt.figure()
sns.barplot(x=default_rates.index, y=default_rates.values)
plt.xlabel("Cluster")
plt.ylabel("Tasa promedio de morosidad (TARGET=1)")
plt.title("Riesgo promedio por cluster")
plt.show()


In [ ]:
# ============================================================
# 8. Visualización de clusters con PCA (2 componentes)
# ============================================================

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(df_scaled)

df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_pca["CLUSTER"] = df["CLUSTER"].values

plt.figure(figsize=(10, 7))
for c in sorted(df_pca["CLUSTER"].unique()):
    subset = df_pca[df_pca["CLUSTER"] == c]
    plt.scatter(subset["PC1"], subset["PC2"], alpha=0.6, label=f"Cluster {c}")

plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Clusters K-Means en espacio PCA (2D)")
plt.legend()
plt.show()

print("Varianza explicada por PC1 y PC2:", pca.explained_variance_ratio_)


In [ ]:
# ============================================================
# 9. Resumen rápido para el informe
# ============================================================

print("===== Resumen Clusters =====\n")
print("Perfiles promedio por cluster:")
display(cluster_profiles)

print("\nTasa de default (TARGET=1) por cluster:")
print(default_rates)

print(f"\nNúmero de clusters usados: {optimal_k}")
print("Varianza explicada total por PC1+PC2:",
      pca.explained_variance_ratio_.sum())


## Interpretación de resultados

Los clusters muestran diferencias claras en variables como ingresos, monto de crédito, ratios de deuda
y antigüedad laboral. Al comparar la tasa de TARGET por cluster, se observan subpoblaciones con mayor
probabilidad de incumplimiento.

Esto indica que K-Means identifica segmentos relevantes que el modelo supervisado podría no capturar
directamente en su estructura.

Clusters con:
- **Altos ratios de crédito/ingreso**
- **Baja antigüedad laboral**
- **Altos ANNUITY_INCOME_RATIO**

tendieron a presentar tasas de TARGET más altas.

Por otro lado, clusters con:
- **Ingresos más altos**
- **Menor endeudamiento relativo**
- **Mayor estabilidad laboral**

mostraron tasas de mora más bajas.


## Relación con el modelo de scoring supervisado

Los clusters obtenidos podrían mejorar el modelo de scoring crediticio mediante:

- Agregar el cluster como variable categórica (cluster_id).
- Detectar segmentos donde el modelo supervisado funciona mejor o peor.
- Identificar posibles sesgos poblacionales.
- Realizar estrategias de riesgo diferenciadas por grupo.

Por tanto, este método sí puede incorporarse en el proyecto final, ya sea como feature o como análisis
complementario de segmentación y monitoreo.
